<div class="admonition danger">
    <p class="admonition-title"></p>
    <p><b>Due to the lapse in federal government funding, NASA is not updating this website. We sincerely regret this inconvenience.</b></p>
</div>

# Accessing via Python

<DIV align="left" style="line-height:1.5em;">
<p>
The following is a collection of Python examples demonstrating how to connect to GIBS access points and exercise various capabilities. Included are examples of how to visualize raster and vector-based data from GIBS, plot imagery on maps, list GIBS capabilities, access GIBS metadata, basic image analysis and more. Please scroll down or use the navigation bar to browse through the examples.
</p>
These examples are also downloadable as a <a href="https://github.com/nasa-gibs/gibs-api-docs/raw/main/docs/python-usage.ipynb">Jupyter Notebook</a>.
</p>
</DIV>

__Import Python Packages And Modules__

<DIV align="left" style="line-height:1.5em;">
<p>
    
Major packages are requests, xml, json, skiimage, matplotlib, cartopy and pillow image.
    
</p>
</DIV>

In [3]:
# install necessary packages for imports
%pip install scikit-image
%pip install scikit-learn
%pip install matplotlib
%pip install cartopy
%pip install folium
%pip install mapbox_vector_tile
%pip install lxml
%pip install pandas
%pip install owslib
%pip install geopandas
%pip install rasterio
%pip install fiona
%pip install ipyleaflet
%pip install cairosvg # If needed, more specific install instructions for cairosvg: https://cairosvg.org/documentation/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.32.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.32.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00


In [4]:
import os
from io import BytesIO
from skimage import io
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import folium
import urllib.request
import urllib.parse
import mapbox_vector_tile
import xml.etree.ElementTree as xmlet
import lxml.etree as xmltree
from PIL import Image as plimg
from PIL import ImageDraw
import numpy as np
import pandas as pd
from owslib.wms import WebMapService
from IPython.display import Image, display
import geopandas as gpd
from shapely.geometry import box
import urllib.request
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
import fiona
from datetime import datetime, timedelta
%matplotlib inline

## OGC Web Map Service (WMS)

Web Map Service (WMS) is the preferred method for accessing static imagery (whereas Web Map Tile Service WMTS is preferred for interactive web maps). For smaller-scale, single image requests, WMS is usually easier to configure than WMTS and can also perform server-side compositing of multiple layers (both vector and raster).

<a id='basic_wms_connection'></a>

### Basic WMS Connection

First we will connect to the GIBS WMS Service and visualize the MODIS_Terra_CorrectedReflectance_TrueColor layer.

In [5]:
# Connect to GIBS WMS Service
wms = WebMapService('https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?', version='1.1.1')

# Configure request for MODIS_Terra_CorrectedReflectance_TrueColor
img = wms.getmap(layers=['GPM_3IMERGDL_Precipitation_Rate'],  # Layers
                 srs='epsg:3857',  # Map projection
                 bbox=(-180,-90,180,90),  # Bounds
                 size=(1200, 600),  # Image size
                 time='2021-09-21',  # Time of data
                 format='image/png',  # Image format
                 transparent=True)  # Nodata transparency

# Save output PNG to a file
out = open('/MODIS_Terra_CorrectedReflectance_TrueColor.png', 'wb')
out.write(img.read())
out.close()

# View image
Image('/MODIS_Terra_CorrectedReflectance_TrueColor.png')

ServiceException: msWMSLoadGetMapParams(): WMS server error. Invalid layer(s) given in the LAYERS parameter. A layer might be disabled for this request. Check wms/ows_enable_request settings.

### Get WMS Capabilities

<DIV align="left" style="line-height:1.5em;">
<p>
    
For WMS, first we want to access the "GetCapabilities" document . GIBS provides four map projections, so there are four WMS endpoints GetCapabilities:
    
Geographic - EPSG:4326: https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi
    
Web Mercator - EPSG:3857: https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi
    
Arctic polar stereographic - EPSG:3413: https://gibs.earthdata.nasa.gov/wms/epsg3413/best/wms.cgi
    
Antarctic polar stereographic - EPSG:3031: https://gibs.earthdata.nasa.gov/wms/epsg3031/best/wms.cgi
     
The code below will show how to get capabilities.
</p>
</DIV>

In [17]:
# Construct capability URL.
wmsUrl = 'https://gibs.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?\
SERVICE=WMtS&REQUEST=GetCapabilities'

# Request WMS capabilities.
response = requests.get(wmsUrl)

# Display capabilities XML in original format. Tag and content in one line.
WmsXml = xmltree.fromstring(response.content)
# print(xmltree.tostring(WmsXml, pretty_print = True, encoding = str))

<a id='display_wms_all_layers'></a>

### Display WMS All Layers

<DIV align="left" style="line-height:1.5em;">
<p>
    
Parse WMS capabilities XML to get total number of layers and display all layer names.
    
</p>
</DIV>

In [19]:
# Currently total layers are 1081.

# Coverts response to XML tree.
WmsTree = xmlet.fromstring(response.content)

alllayer = []
layerNumber = 0

# Parse XML.
for child in WmsTree.iter():
    for layer in child.findall("./{http://www.opengis.net/wmts}Capability/{http://www.opengis.net/wmts}Layer//*/"):
         if layer.tag == '{http://www.opengis.net/wmts}Layer':
            f = layer.find("{http://www.opengis.net/wmts}Name")
            if f is not None:
                alllayer.append(f.text)

                layerNumber += 1

print('There are layers: ' + str(layerNumber))

for one in sorted(alllayer)[:5]:
    print(one)
print('...')
for one in sorted(alllayer)[-5:]:
    print(one)

There are layers: 0
...


### Search WMS Layer And Its Attributes

<DIV align="left" style="line-height:1.5em;">
<p>
    
Requesting WMS data needs layer name, bounding box, time, projection, data format and so on. Enter a layer
name to search its attributes.
    
</p>
</DIV>

In [14]:
# Define layername to use.
layerName = 'VIIRS_NOAA20_Flood_Map'

# Get general information of WMS.
for child in WmsTree.iter():
    if child.tag == '{http://www.opengis.net/wms}WMS_Capabilities':
        print('Version: ' +child.get('version'))

    if child.tag == '{http://www.opengis.net/wms}Service':
        print('Service: ' +child.find("{http://www.opengis.net/wms}Name").text)

    if child.tag == '{http://www.opengis.net/wms}Request':
        print('Request: ')
        for e in child:
            print('\t ' + e.tag.partition('}')[2])

        all = child.findall(".//{http://www.opengis.net/wms}Format")
        if all is not None:
            print("Format: ")
            for g in all:
                print("\t " + g.text)

        for e in child.iter():
            if e.tag == "{http://www.opengis.net/wms}OnlineResource":
                print('URL: ' + e.get('{http://www.w3.org/1999/xlink}href'))
                break

# Get layer attributes.
for child in WmsTree.iter():
    for layer in child.findall("./{http://www.opengis.net/wms}Capability/{http://www.opengis.net/wms}Layer//*/"):
         if layer.tag == '{http://www.opengis.net/wms}Layer':
            f = layer.find("{http://www.opengis.net/wms}Name")
            if f is not None:
                if f.text == layerName:
                    # Layer name.
                    print('Layer: ' + f.text)

                    # All elements and attributes:
                    # CRS
                    e = layer.find("{http://www.opengis.net/wms}CRS")
                    if e is not None:
                        print('\t CRS: ' + e.text)

                    # BoundingBox.
                    e = layer.find("{http://www.opengis.net/wms}EX_GeographicBoundingBox")
                    if e is not None:
                        print('\t LonMin: ' + e.find("{http://www.opengis.net/wms}westBoundLongitude").text)
                        print('\t LonMax: ' + e.find("{http://www.opengis.net/wms}eastBoundLongitude").text)
                        print('\t LatMin: ' + e.find("{http://www.opengis.net/wms}southBoundLatitude").text)
                        print('\t LatMax: ' + e.find("{http://www.opengis.net/wms}northBoundLatitude").text)

                    # Time extent.
                    e = layer.find("{http://www.opengis.net/wms}Dimension")
                    if e is not None:
                        print('\t TimeExtent: ' + e.text)

                    # Style.
                    e = layer.find("{http://www.opengis.net/wms}Style")
                    if e is not None:
                        f = e.find("{http://www.opengis.net/wms}Name")
                        if f is not None:
                            print('\t Style: ' + f.text)

print('')

Version: 1.3.0
Service: WMS
Request: 
	 GetCapabilities
	 GetMap
Format: 
	 text/xml
	 image/png
	 image/jpeg
	 application/vnd.google-earth.kml.xml
	 application/vnd.google-earth.kmz
	 image/png; mode=8bit
	 image/vnd.jpeg-png
	 image/vnd.jpeg-png8
	 application/x-pdf
	 image/svg+xml
	 image/tiff
	 application/json
URL: https://gitc.earthdata.nasa.gov/wms/epsg3857/best/?



## OGC Web Map Tile Service (WMTS)

Web Map Tile Service (WMTS) is normally used for interactive web mapping, but may be used for general visualizations and data analysis. WMTS is much more responsive for interactive maps and very scalable for generating large images or bulk downloads, but compared to WMS, it is more challenging to configure if you just need a single, reasonably-sized image.

### Get WMTS Capabilities

<DIV align="left" style="line-height:1.5em;">
<p>
    
This example shows how to get WMTS capabilities and display the GetCapabilities XML content.
    
</p>
</DIV>

In [6]:
# Construct WMTS capability URL.
wmtsUrl = 'http://gibs.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?SERVICE=WMTS&REQUEST=GetCapabilities'

# Request capabilities.
response = requests.get(wmtsUrl)

# Display capability XML.
WmtsXml = xmltree.fromstring(response.content)

# Uncomment the following to display the large file:
# print(xmltree.tostring(WmtsXml, pretty_print = True, encoding = str))

<a id='display_wmts_all_layers'></a>

### Display All Layers of WMTS Capabilities.

<DIV align="left" style="line-height:1.5em;">
<p>
    
This example shows how to parse the WMTS GetCapabilities document and print the names of all of its layers.
    
</p>
</DIV>

In [7]:
# Convert capability response to XML tree.
WmtsTree = xmlet.fromstring(response.content)

alllayer = []
layerNumber = 0

# Parse capability XML tree.
for child in WmtsTree.iter():
    for layer in child.findall("./{http://www.opengis.net/wmts/1.0}Layer"):
         if '{http://www.opengis.net/wmts/1.0}Layer' == layer.tag:
            f=layer.find("{http://www.opengis.net/ows/1.1}Identifier")
            if f is not None:
                alllayer.append(f.text)
                layerNumber += 1

# Print the first five and last five layers.
print('Number of layers: ', layerNumber)
for one in sorted(alllayer)[:5]:
    print(one)
print('...')
for one in sorted(alllayer)[-5:]:
    print(one)

Number of layers:  1225
AIRS_L2_Carbon_Monoxide_500hPa_Volume_Mixing_Ratio_Day
AIRS_L2_Carbon_Monoxide_500hPa_Volume_Mixing_Ratio_Night
AIRS_L2_Cloud_Top_Height_Day
AIRS_L2_Cloud_Top_Height_Night
AIRS_L2_Dust_Score_Day
...
VIIRS_SNPP_SurfaceReflectance_BandsM11-M7-M5
VIIRS_SNPP_SurfaceReflectance_BandsM5-M4-M3
VIIRS_SNPP_Thermal_Anomalies_375m_All
VIIRS_SNPP_Thermal_Anomalies_375m_Day
VIIRS_SNPP_Thermal_Anomalies_375m_Night


In [23]:
word_to_search = "Aerosol"

# Namespace prefixes for cleaner code
ns = {
    "wmts": "http://www.opengis.net/wmts/1.0",
    "ows": "http://www.opengis.net/ows/1.1"
}

matching_layers = []

for layer in WmtsTree.findall(".//wmts:Layer", ns):
    identifier = layer.find("ows:Identifier", ns)
    if identifier is not None and word_to_search in identifier.text:
        matching_layers.append(identifier.text)

# Print or return matching layers
if matching_layers:
    print("Matching layers:")
    for name in matching_layers:
        print("-", name)
else:
    print("No layers found containing:", word_to_search)


Matching layers:
- SWDB_Aerosol_Angstrom_Exponent_Daily
- SWDB_Aerosol_Angstrom_Exponent_Monthly
- OMPS_NOAA21_LimbProfiler_Aerosol_ExinctionCoefficient_12KM
- OMPS_SNPP_LimbProfiler_Aerosol_ExinctionCoefficient_12KM
- OMPS_NOAA21_LimbProfiler_Aerosol_ExinctionCoefficient_14KM
- OMPS_SNPP_LimbProfiler_Aerosol_ExinctionCoefficient_14KM
- OMPS_NOAA21_LimbProfiler_Aerosol_ExinctionCoefficient_16KM
- OMPS_SNPP_LimbProfiler_Aerosol_ExinctionCoefficient_16KM
- OMPS_NOAA21_LimbProfiler_Aerosol_ExinctionCoefficient_18KM
- OMPS_SNPP_LimbProfiler_Aerosol_ExinctionCoefficient_18KM
- OMPS_NOAA21_LimbProfiler_Aerosol_ExinctionCoefficient_20KM
- OMPS_SNPP_LimbProfiler_Aerosol_ExinctionCoefficient_20KM
- OMPS_NOAA21_LimbProfiler_AerosolHeight
- OMPS_SNPP_LimbProfiler_AerosolHeight
- OMPS_NOAA20_NadirMapper_AerosolIndex_360
- OMPS_NOAA21_NadirMapper_AerosolIndex_360
- OMPS_NOAA20_NadirMapper_AerosolIndex_380
- OMPS_NOAA21_NadirMapper_AerosolIndex_380
- OMI_Aerosol_Index
- OMPS_Aerosol_Index
- OMPS_NOA

### Search WMTS Vector Layer, Attributes And Vector Information

<DIV align="left" style="line-height:1.5em;">
<p>
    
This example shows how to search a WMTS layer and to parse its attributes and vector information.
    
</p>
</DIV>

In [28]:
# Get general information of WMTS from XML tree.
for child in WmtsTree.iter():
    if child.tag == '{http://www.opengis.net/wmts/1.0}Capabilities':
        print('Version: ' + child.get('version'))

    if child.tag == '{http://www.opengis.net/ows/1.1}ServiceType':
        print('Service: ' + child.text)

    if child.tag == '{http://www.opengis.net/ows/1.1}OperationsMetadata':
        print('Request: ')
        for e in child:
            print('\t ' + e.get('name'))

# Parse layer attributes and vector information.
for child in WmtsTree.iter():
    for layer in child.findall("./{http://www.opengis.net/wmts/1.0}Layer"):
         if '{http://www.opengis.net/wmts/1.0}Layer' == layer.tag:
            f = layer.find("{http://www.opengis.net/ows/1.1}Identifier")
            if f is not None:
                if f.text == 'SMAP_L3_Soil_Moisture':
                    # Layer name.
                    print('Layer: ' + f.text)

                    # All elements and attributes:

                    # BoundingBox.
                    e = layer.find("{http://www.opengis.net/ows/1.1}WGS84BoundingBox")
                    if e is not None:
                        print("\t crs: " + e.get('crs'))
                        print("\t UpperCorner: " + e.find("{http://www.opengis.net/ows/1.1}UpperCorner").text)
                        print("\t LowerCorner: " + e.find("{http://www.opengis.net/ows/1.1}LowerCorner").text)

                    # TileMatrixSet.
                    e = layer.find("{http://www.opengis.net/wmts/1.0}TileMatrixSetLink")
                    if e is not None:
                        print("\t TileMatrixSet: " + e.find("{http://www.opengis.net/wmts/1.0}TileMatrixSet").text)

                    # Time extent.
                    e = layer.find("{http://www.opengis.net/wmts/1.0}Dimension")
                    if e is not None:
                        all = e.findall("{http://www.opengis.net/wmts/1.0}Value")
                        if all is not None:
                            print("\t TimeExtent: ")
                            for g in all:
                                print("\t\t " + g.text)

                    # Format.
                    e = layer.find("{http://www.opengis.net/wmts/1.0}Format")
                    if e is not None:
                        print("\t Format: " + e.text)

                    # Style.
                    e = layer.find("{http://www.opengis.net/wmts/1.0}Style")
                    if e is not None:
                        g=e.find("{http://www.opengis.net/ows/1.1}Identifier")
                        if g is not None:
                            print("\t Style: " + g.text)

                    # Template.
                    e = layer.find("{http://www.opengis.net/wmts/1.0}ResourceURL")
                    if e is not None:
                        print("\t Template: " + e.get('template'))

                    # Vector metadata.
                    for e in layer.findall("{http://www.opengis.net/ows/1.1}Metadata"):
                        if "vector-metadata" in e.get("{http://www.w3.org/1999/xlink}href"):
                            vectorMetadata=e.get("{http://www.w3.org/1999/xlink}href")
                            print('\t Vector metadata: ' + vectorMetadata)

                            response = urllib.request.urlopen(vectorMetadata)

                            # Load to json.
                            data = json.loads(response.read())

                            # Parse json.
                            for p in data['mvt_properties']:
                                keys = list(p.keys())
                                if 'Identifier' in keys:
                                    print('\t\t Identifier: ' + p['Identifier'])
                                if 'Title' in keys:
                                    print('\t\t Title: ' + p['Title'])
                                if 'Description' in keys:
                                    print('\t\t Description: ' + p['Description'])
                                if 'Units' in keys:
                                    print('\t\t Units: ' + p['Units'])
                                if 'DataType' in keys:
                                    print('\t\t DataType: ' + p['DataType'])
                                if 'ValueRanges' in keys:
                                    print('\t\t ValueRanges: ' + str(p['ValueRanges']))
                                if 'ValueMap' in keys:
                                    print('\t\t ValueMap: ' + str(p['ValueMap']))
                                if 'Function' in keys:
                                    print('\t\t Function: ' + p['Function'])
                                if 'IsOptional' in keys:
                                    print('\t\t IsOptional: ' + str(p['IsOptional']))
                                if 'IsLabel' in keys:
                                    print('\t\t IsLabel: ' + str(p['IsLabel']))

                                print('\n')

                            # There two vector metadata. Only need one, so break.
                            break

print('')

Version: 1.0.0
Service: OGC WMTS
Request: 
	 GetCapabilities
	 GetTile



### Read WMTS Vector Data

<DIV align="left" style="line-height:1.5em;">
<p>
    
This example shows how to get WMTS vector data from a Mapbox Vector Tile (MVT). Also shows how to parse vector data values.
    
</p>
</DIV>

In [ ]:
# Vector data format.
'''
{
   'MODIS_Aqua_Thermal_Anomalies_All':
   {
    'extent': 4096,
    'version': 1,
    'features':
    [
        {'geometry':
            {'type': 'Point',
             'coordinates': [4028, 3959]},
             'properties': {'LATITUDE': 35.397,
                            'LONGITUDE': -90.3,
                            'BRIGHTNESS': 307.3,
                            'SCAN': 3.2,
                            'TRACK': 1.7,
                            'ACQ_DATE': '2020-10-01',
                            'ACQ_TIME': '18:30',
                            'SATELLITE': 'A',
                            'CONFIDENCE': 48,
                            'VERSION': '6.0NRT',
                            'BRIGHT_T31': 296.0,
                            'FRP': 21.4,
                            'DAYNIGHT': 'D',
                            'UID': 13159},
                            'id': 0,
                            'type': 1
                           }
            }
        }
        ,,,
    ]
}
'''

# Below both kvp and restful methods work.
'''
kvp = 'https://gibs.earthdata.nasa.gov/wmts/epsg4326/best/wmts.cgi?\
TIME=2020-10-01T00:00:00Z&FORMAT=application/vnd.mapbox-vector-tile&\
layer=MODIS_Aqua_Thermal_Anomalies_All&tilematrixset=1km&\
Service=WMTS&Request=GetTile&Version=1.0.0&TileMatrix=4&TileCol=3&TileRow=3'
response = requests.get(kvp)
'''

restful = 'https://gibs.earthdata.nasa.gov/wmts/epsg4326/best/MODIS_Aqua_Thermal_Anomalies_All\
/default/2020-10-01T00:00:00Z/1km/4/3/4.mvt'

# Request data.
response = requests.get(restful)

# Parse vector values.
data = response.content
dataDictionary = mapbox_vector_tile.decode(data)
for key in dataDictionary.keys():
    parameterDictionary = dataDictionary[key]
    features = parameterDictionary['features']
    # Print vector data format.
    #print(features)

    lat = []
    lon = []
    brightness = []
    for f in features:
        p = f['properties']
        lat.append(p['LATITUDE'])
        lon.append(p['LONGITUDE'])
        brightness.append(p['BRIGHTNESS'])

print('lat number: ' + str(len(lat)))
print(str(lat))
print('lon number: ' + str(len(lon)))
print(str(lon))
print('brightness number: ' + str(len(brightness)))
print('brightness min: ' + str(min(brightness)))
print('brightness min: ' + str(max(brightness)))
print(str(brightness))

print('')

lat number: 81
[35.397, 35.403, 35.405, 35.446, 35.45, 35.454, 35.467, 35.47, 35.484, 36.053, 33.225, 33.449, 33.45, 33.451, 33.451, 33.625, 33.627, 33.755, 33.756, 33.77, 33.8, 33.803, 33.853, 33.853, 33.855, 33.867, 34.076, 34.346, 34.356, 34.457, 31.032, 31.034, 31.046, 31.048, 31.365, 31.608, 31.89, 31.892, 31.899, 18.316, 19.026, 19.091, 19.094, 19.232, 19.592, 19.594, 19.639, 19.653, 19.8, 19.808, 19.961, 20.518, 20.601, 20.611, 20.723, 20.725, 21.151, 21.726, 21.728, 21.911, 22.04, 22.917, 23.775, 25.189, 25.191, 25.228, 25.723, 25.843, 27.436, 27.447, 28.799, 28.801, 28.914, 29.095, 29.096, 29.527, 29.529, 29.529, 29.531, 29.561, 29.805]
lon number: 81
[-90.3, -90.272, -90.266, -90.676, -90.67, -90.641, -92.205, -92.215, -92.209, -89.904, -91.815, -94.62, -94.167, -94.593, -94.589, -93.99, -93.982, -94.507, -94.516, -94.509, -93.806, -93.798, -94.621, -94.626, -94.594, -94.627, -96.957, -91.197, -91.157, -91.021, -95.208, -95.182, -95.209, -95.183, -98.348, -95.122, -90.842, -9